# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [64]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [65]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [66]:
train_data,test_data = sales.random_split(.8,seed=0)

In [85]:
train_data.head()


id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [86]:
test_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [69]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.080326     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [70]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [71]:
example_predictions = example_model.predict(train_data)
print example_predictions # should be 271789.505878

[271789.5058780203, 718880.3134862408, 207554.31080448133, 466612.01434113237, 436435.24634671607, 1568323.948940347, 449210.3718142016, 237413.10310809387, 461031.5702092758, 506141.9789589803, 1032865.724680975, 330561.65261979745, 336924.17472740496, 347277.26251285, 502669.9688626556, 339178.7353570801, 343177.79024188116, 293867.74671666673, 350694.81931260903, 873745.97504757, 495834.85526313726, 305647.35306643386, 681031.9006199897, 677559.890523665, 341178.2627994806, 384234.6908102188, 655536.1029815841, 343495.76593268494, 574631.2621045948, 496416.35334817925, 635457.3895853382, 358102.94179020566, 585829.37036932, 219325.42794728486, 655536.1029815841, 915480.4190428223, 394796.8476933364, 303329.8499332295, 906704.3484601378, 211862.85217312266, 567223.1396269982, 299075.7618611539, 778335.1481554362, 274943.5402835412, 773389.8829489788, 349540.3123494886, 594400.4890170005, 639184.4328689375, 737540.9975251282, 673305.8024515894, 810138.2421486751, 431917.6358804022, 58

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [72]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    #print(predictions)
    
    # Then compute the residuals/errors
    errors = (outcome - predictions)
        
    # Then square and add them up
    RSS = (errors**2).sum()
    
    return RSS

Test your function by computing the RSS on TEST data for the example model:

In [73]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [74]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [75]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [76]:
# create the remaining 3 features in both TEST and TRAIN data

# bed_bath_rooms = bedrooms*bathrooms
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

# log_sqft_living = log(sqft_living)
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

# lat_plus_long = lat + long As an example here's the first one:
train_data['lat_plus_long'] = train_data['long'] + train_data['lat']
test_data['lat_plus_long'] = test_data['long'] + test_data['lat']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [83]:
"""
quiz_1_model = graphlab.linear_regression.create(train_data, target = 'price', features = quiz_1_features, 
                                                  validation_set = None)
quiz_1_predictions = quiz_1_model.predict(test_data)
quiz_1_answer = round(quiz_1_predictions.mean(), 2)
print quiz_1_answer
"""

quiz_1_features = ['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']

bedrooms_squared_mean_test = test_data['bedrooms_squared'].mean()
bed_bath_rooms_mean_test = test_data['bed_bath_rooms'].mean()
log_sqft_living_mean_test = test_data['log_sqft_living'].mean()
lat_plus_long_mean_test = test_data['lat_plus_long'].mean()

bedrooms_squared_mean_train = train_data['bedrooms_squared'].mean()
bed_bath_rooms_mean_train = train_data['bed_bath_rooms'].mean()
log_sqft_living_mean_train = train_data['log_sqft_living'].mean()
lat_plus_long_mean_train = train_data['lat_plus_long'].mean()

print round(bedrooms_squared_mean_test, 2)
print round(bed_bath_rooms_mean_test, 2)
print round(log_sqft_living_mean_test, 2)
print round(lat_plus_long_mean_test, 2)

print round(bedrooms_squared_mean_train, 2)
print round(bed_bath_rooms_mean_train, 2)
print round(log_sqft_living_mean_train, 2)
print round(lat_plus_long_mean_train, 2)

12.45
7.5
7.55
-74.65
12.17
7.5
7.55
-74.65


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [78]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [79]:
# Learn the three models: (don't forget to set validation_set = None)

model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                  validation_set = None)

model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                                  validation_set = None)

model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.064109     | 3193229.177894     | 228200.043155 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [80]:
# Examine/extract each model's coefficients:
model_1_coeff = model_1.get('coefficients')
model_2_coeff = model_2.get('coefficients')
model_3_coeff = model_3.get('coefficients')

print(model_1_coeff, model_2_coeff, model_3_coeff)

(Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 6

Data:
+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56140675.7444 | 1649985.42028 |
| sqft_living |  None | 310.263325778  | 3.18882960408 |
|   bedrooms  |  None | -59577.1160682 | 2487.27977322 |
|  bathrooms  |  None | 13811.8405418  | 3593.54213297 |
|     lat     |  None | 629865.789485  | 13120.7100323 |
|     long    |  None | -214790.285186 | 13284.2851607 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]
, Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 7

Data:
+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54410676.1152 | 1650405.16541 |
|  sqft_living   |  

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

* the coefficient for 'bathrooms' in model 1 >>> 
13811.8405418, positive.


* the coefficient for 'bathrooms' in model 2 >>>
-77972.3305135, negative.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [81]:
# Compute the RSS on TRAINING data for each of the three models and record the values:

"""
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
"""

rss_model_1 = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

rss_list_train = [rss_model_1, rss_model_2, rss_model_3]

print(rss_model_1, rss_model_2, rss_model_3)
print(rss_list_train.index(min(rss_list_train)))

(971328233543667.1, 961592067855751.0, 905276314555406.0)
2


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

* model with the lowest RSS : model 3 / Yes, I expected this because it has the most features among 3 models.

Now compute the RSS on on TEST data for each of the three models.

In [82]:
# Compute the RSS on TESTING data for each of the three models and record the values:

rss_model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

rss_list_test = [rss_model_1_test, rss_model_2_test, rss_model_3_test]

print(rss_model_1_test, rss_model_2_test, rss_model_3_test)
print(rss_list_test.index(min(rss_list_test)))

(226568089092795.38, 224368799993615.4, 251829318951767.94)
1


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

* model with the lowest RSS : model 2 / No. 
* I think that by adding lat_plus_long feature the final estimates are calculated more correctly preventing any over/underestimation caused by using only 'lat' and 'long' as in model 2.


Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.

bedrooms times bathrooms gives what's called an "interaction" feature. It is large when both of them are large.

Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.

Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)
